In [1]:
path=r"/content/twitter_training.csv"

In [2]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences

In [4]:
df=pd.read_csv(path,nrows=50)

In [5]:
df.head(10)

,Sentiments,Tweets
0,Positive,I am coming to the borders and I will kill you...
1,Negative,The Worst Way to Die in
2,Positive,im coming on borderlands and i will murder you...
3,Negative,All the Borderlands are damn rubbish
4,Positive,im getting into borderlands and i can murder y...
5,Positive,So I spent a few hours making something for fu...
6,Negative,Saw ran an ad for the new borderlands dlc s st...
7,Positive,So I spent a few hours doing something for fun...
8,Negative,Digital Foundry is mostdef full of shit.. bord...
9,Positive,2010 So I spent a few hours making something f...


In [6]:
#replace
df["Sentiments"].replace({"Positive":1,"Negative":0},inplace=True)
df.head(10)

,Sentiments,Tweets
0,1,I am coming to the borders and I will kill you...
1,0,The Worst Way to Die in
2,1,im coming on borderlands and i will murder you...
3,0,All the Borderlands are damn rubbish
4,1,im getting into borderlands and i can murder y...
5,1,So I spent a few hours making something for fu...
6,0,Saw ran an ad for the new borderlands dlc s st...
7,1,So I spent a few hours doing something for fun...
8,0,Digital Foundry is mostdef full of shit.. bord...
9,1,2010 So I spent a few hours making something f...


In [7]:
x = np.array(df["Tweets"].values)
y = np.array(df["Sentiments"].values)

In [8]:
x_filtered = []

for tweet in x:
    # Check if tweet is a string
    if isinstance(tweet, str):
        # Lowercasing the sentence
        tweet = tweet.lower()
        tweet_filtered = ""
        for char in tweet:
            if char not in ''' !()-[]{};:'"\,<>./?@#$%^&*_~''':
                tweet_filtered += char
            else:
                tweet_filtered += ' '  # Add space between words
        x_filtered.append(tweet_filtered)

print(x_filtered)


['i am coming to the borders and i will kill you all ', 'the worst way to die in', 'im coming on borderlands and i will murder you all ', 'all the borderlands are damn rubbish', 'im getting into borderlands and i can murder you all ', 'so i spent a few hours making something for fun      if you don t know i am a huge  borderlands fan and maya is one of my favorite characters  so i decided to make myself a wallpaper for my pc    here is the original image versus the creation i made    enjoy  pic twitter com mlsi5wf9jg', 'saw ran an ad for the new borderlands dlc s stuff and i just half got sad happy again knowing fully well bl3 was likely so horribly mediocre i might already miss the bl2', 'so i spent a few hours doing something for fun    if you don t know i m a huge   borderlands fan and maya is one of my favorite characters ', 'digital foundry is mostdef full of shit   borderlands 3 pc system requirements and recommended are considerably pc hardware garbage   7+ years ', '2010 so i s

In [9]:
# converting sentences to vectors

vocabulary_size = 5000

one_hot_encoded = [one_hot(Tweets,vocabulary_size) for Tweets in x_filtered]

one_hot_encoded

[[3524, 804, 1090, 473, 1513, 123, 4942, 3524, 916, 4715, 1245, 1115],
 [1513, 4035, 2818, 473, 4449, 14],
 [2815, 1090, 3667, 2031, 4942, 3524, 916, 3561, 1245, 1115],
 [1115, 1513, 2031, 169, 831, 3016],
 [2815, 844, 2010, 2031, 4942, 3524, 3382, 3561, 1245, 1115],
 [2625,
  3524,
  2585,
  3898,
  3355,
  2881,
  2431,
  4914,
  2619,
  2071,
  3832,
  1245,
  3376,
  1671,
  4041,
  3524,
  804,
  3898,
  2022,
  2031,
  1594,
  4942,
  3028,
  3233,
  2532,
  2149,
  4212,
  2583,
  3250,
  2625,
  3524,
  1927,
  473,
  3068,
  975,
  3898,
  1379,
  2619,
  4212,
  4388,
  4347,
  3233,
  1513,
  2247,
  2032,
  1359,
  1513,
  3824,
  3524,
  3433,
  4923,
  84,
  2610,
  4773,
  2480],
 [4558,
  4383,
  2422,
  4337,
  2619,
  1513,
  1416,
  2031,
  4314,
  2856,
  4715,
  4942,
  3524,
  2575,
  2861,
  1924,
  1195,
  4396,
  1356,
  1377,
  1549,
  1138,
  532,
  3561,
  1415,
  2625,
  1538,
  4651,
  3524,
  1746,
  4938,
  4607,
  1513,
  4544],
 [2625,
  3524,
  2585,


In [10]:
# padding the sentences

max_lenght = 500

x_padded = pad_sequences(one_hot_encoded,max_lenght,padding = "post")

In [15]:
# Splitting the dataset

from sklearn.model_selection import train_test_split

# Print the lengths of x_padded_trimmed and y before trimming
print("Length of y before trimming:", len(y))

# Trim y to match the length of x_padded_trimmed
# Replace x_padded_trimmed with a variable that is defined in the code
y = y[:len(x_padded)]

# Print the lengths of x_padded_trimmed and y after trimming
print("Length of x_padded_trimmed after trimming:", len(x_padded))
print("Length of y after trimming:", len(y))

# Split the dataset into training and testing sets
# Use a different splitting method that can handle datasets with different lengths
x_train, x_test, y_train, y_test = train_test_split(x_padded, y, test_size=0.3, shuffle=False)

Length of y before trimming: 50
Length of x_padded_trimmed after trimming: 50
Length of y after trimming: 50


In [16]:
# model creation
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding

embedded_vector_size = 35
model = Sequential()
model.add(Embedding(vocabulary_size,embedded_vector_size,input_length=max_lenght))
model.add(LSTM(100))
model.add(Dense(1,activation="sigmoid"))

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 35)           175000    
                                                                 
 lstm (LSTM)                 (None, 100)               54400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 229501 (896.49 KB)
Trainable params: 229501 (896.49 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
# training the model


y_train_binary = np.zeros_like(y_train)
y_train_binary[y_train != 'Neutral'] = 1
y_train = y_train_binary.astype('float32')
model.compile(loss="binary_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, epochs=10)

Epoch 1/10


In [ ]:
score = model.evaluate(x_test,y_test)
print("ACCURACY : " , score)